In [1]:
include("data_handler.jl")

read_dataset (generic function with 2 methods)

In [2]:
trnb, devb, tstb, vocabsize = read_dataset("../data/2016-NAACL/STxyz/", 1; addworld=false);

In [17]:
trn_sources = hcat(map(x->reshape(x.sources, 3, 1), trnb)...);
trn_targets = hcat(map(x->reshape(x.targets, 3, 1), trnb)...);
dev_sources = hcat(map(x->reshape(x.sources, 3, 1), devb)...);
dev_targets = hcat(map(x->reshape(x.targets, 3, 1), devb)...);
test_sources = hcat(map(x->reshape(x.sources, 3, 1), tstb)...);
test_targets = hcat(map(x->reshape(x.targets, 3, 1), tstb)...);

In [23]:
println("*** Averages ***")
println("Source Block Train μ: $(mean(trn_sources, 2)) σ: $(std(trn_sources, 2))")
println("Target Position Train μ: $(mean(trn_targets, 2)) σ: $(std(trn_targets, 2))")
println("")
println("Source Block Dev μ: $(mean(dev_sources, 2)) σ: $(std(dev_sources, 2))")
println("Target Position Dev μ: $(mean(dev_targets, 2)) σ: $(std(dev_targets, 2))")
println("")
println("Source Block Test μ: $(mean(test_sources, 2)) σ: $(std(test_sources, 2))")
println("Target Position Test μ: $(mean(test_targets, 2)) σ: $(std(test_targets, 2))")

*** Averages ***
Source Block Train μ: Float32[-0.0138969; 0.099988; -0.00856706] σ: Float32[0.621981; 1.20259f-5; 0.596289]
Target Position Train μ: Float32[0.129143; 0.099988; -0.124375] σ: Float32[0.323372; 1.20259f-5; 0.447791]

Source Block Dev μ: Float32[-0.0437697; 0.100001; -0.00277486] σ: Float32[0.580318; 7.52726f-7; 0.604469]
Target Position Dev μ: Float32[0.16377; 0.100001; -0.134607] σ: Float32[0.319323; 7.52726f-7; 0.443743]

Source Block Test μ: Float32[0.00186968; 0.100003; 0.00371109] σ: Float32[0.620826; 3.19679f-6; 0.588454]
Target Position Test μ: Float32[0.127507; 0.100003; -0.113343] σ: Float32[0.298691; 3.19679f-6; 0.452174]


In [30]:
μsource = mean(trn_sources, 2)
μtarget = mean(trn_targets, 2)
dev_source_baseline = map(x-> sqrt(sum((μsource .- dev_sources[:, x]) .^ 2)) / 0.1524, 1:size(dev_sources, 2))
dev_target_baseline = map(x-> sqrt(sum((μtarget .- dev_targets[:, x]) .^ 2)) / 0.1524, 1:size(dev_targets, 2))
test_source_baseline = map(x-> sqrt(sum((μsource .- test_sources[:, x]) .^ 2)) / 0.1524, 1:size(test_sources, 2))
test_target_baseline = map(x-> sqrt(sum((μtarget .- test_targets[:, x]) .^ 2)) / 0.1524, 1:size(test_targets, 2))

println("** Baseline (Train Mean) (Median, Mean) **")
println("Dev Source: $((median(dev_source_baseline), mean(dev_source_baseline))) Target: $((median(dev_target_baseline), mean(dev_target_baseline)))")
println("Test Source: $((median(test_source_baseline), mean(test_source_baseline))) Target: $((median(test_target_baseline), mean(test_target_baseline)))")

** Baseline (Train Mean) (Median, Mean) **
Dev Source: (5.460992416371823, 5.176767081663597) Target: (3.463604397035333, 3.338602757283673)
Test Source: (5.440029494092846, 5.30981977431434) Target: (3.463604397035333, 3.27121680565107)


In [44]:
trnsc = Dict();
devsc = Dict();
testsc = Dict();
for x in vcat(map(x->x.sourceinds, trnb)...); trnsc[x] = get!(trnsc, x, 0) + 1; end;
for x in vcat(map(x->x.sourceinds, devb)...); devsc[x] = get!(trnsc, x, 0) + 1; end;
for x in vcat(map(x->x.sourceinds, tstb)...); testsc[x] = get!(trnsc, x, 0) + 1; end;

In [51]:
using Plots
plotlyjs()
x = collect(keys(trnsc))
y = collect(values(trnsc))
pie(x, y, title="Train Source Class Distribution")

In [52]:
x = collect(keys(devsc))
y = collect(values(devsc))
pie(x, y, title="Dev Source Class Distribution")

In [53]:
x = collect(keys(testsc))
y = collect(values(testsc))
pie(x, y, title="Test Source Class Distribution")